# Test sample_weight, kernels, C, sklearn estimator

# Setup
Uncomment the next cell if STree is not already installed

In [ ]:
#
# Google Colab setup
#
#!pip install git+https://github.com/doctorado-ml/stree
!pip install pandas

In [ ]:
import time
import random
import warnings
import os
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.estimator_checks import check_estimator
from sklearn.datasets import make_classification, load_iris, load_wine
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.exceptions import ConvergenceWarning
from stree import Stree
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
if not os.path.isfile('data/creditcard.csv'):
    !wget --no-check-certificate --content-disposition http://nube.jccm.es/index.php/s/Zs7SYtZQJ3RQ2H2/download
    !tar xzf creditcard.tgz

In [ ]:
random_state=1

def load_creditcard(n_examples=0):
    df = pd.read_csv('data/creditcard.csv')
    print("Fraud: {0:.3f}% {1}".format(df.Class[df.Class == 1].count()*100/df.shape[0], df.Class[df.Class == 1].count()))
    print("Valid: {0:.3f}% {1}".format(df.Class[df.Class == 0].count()*100/df.shape[0], df.Class[df.Class == 0].count()))
    y = df.Class
    X = df.drop(['Class', 'Time', 'Amount'], axis=1).values
    if n_examples > 0:
        # Take first n_examples samples
        X = X[:n_examples, :]
        y = y[:n_examples, :]
    else:
        # Take all the positive samples with a number of random negatives
        if n_examples < 0:
            Xt = X[(y == 1).ravel()]
            yt = y[(y == 1).ravel()]
            indices = random.sample(range(X.shape[0]), -1 * n_examples)
            X = np.append(Xt, X[indices], axis=0)
            y = np.append(yt, y[indices], axis=0)
    print("X.shape", X.shape, " y.shape", y.shape)
    print("Fraud: {0:.3f}% {1}".format(len(y[y == 1])*100/X.shape[0], len(y[y == 1])))
    print("Valid: {0:.3f}% {1}".format(len(y[y == 0]) * 100 / X.shape[0], len(y[y == 0])))
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=random_state)
    return Xtrain, Xtest, ytrain, ytest

data = load_creditcard(-5000) # Take all true samples with up to 5000 of the others
# data = load_creditcard(5000)  # Take the first 5000 samples
# data = load_creditcard(-1000) # Take 1000 samples

Xtrain = data[0]
Xtest = data[1]
ytrain = data[2]
ytest = data[3]
weights = compute_sample_weight("balanced", ytrain)
weights_test = compute_sample_weight("balanced", ytest)
print(weights[:4], weights_test[:4])

# Tests

## Test sample_weights
Compute accuracy with weights in samples. The weights are set based on the inverse of the number of samples of each class

In [ ]:
C = 23
print("Accuracy of Train without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtrain, ytrain))
print("Accuracy of Train with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtrain, ytrain))
print("Accuracy of Tests without weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain).score(Xtest, ytest))
print("Accuracy of Tests with    weights", Stree(C=C, random_state=1).fit(Xtrain, ytrain, sample_weight=weights).score(Xtest, ytest))

## Test accuracy with different kernels
Compute accuracy on train and test set with default hyperparmeters of every kernel

In [ ]:
random_state=1
for kernel in ['linear', 'rbf', 'poly']:
    now = time.time()
    clf = Stree(C=7, kernel=kernel, random_state=random_state).fit(Xtrain, ytrain)
    accuracy_train = clf.score(Xtrain, ytrain)
    accuracy_test = clf.score(Xtest, ytest)
    time_spent = time.time() - now
    print(f"Time: {time_spent:.2f}s\tKernel: {kernel}\tAccuracy_train: {accuracy_train}\tAccuracy_test: {accuracy_test}")


## Test diferent values of C

In [ ]:
t = time.time()
for C in (.001, .01, 1, 5, 17):
    clf = Stree(C=C, kernel="linear", random_state=random_state)
    clf.fit(Xtrain, ytrain)
    print(f"************** C={C} ****************************")
    print(f"Classifier's accuracy (train): {clf.score(Xtrain, ytrain):.4f}")
    print(f"Classifier's accuracy (test) : {clf.score(Xtest, ytest):.4f}")
    print(clf)
    print(f"**************************************************")
print(f"{time.time() - t:.4f} secs")

## Test iterator
Check different ways of using the iterator

In [ ]:
#check iterator
for i in list(clf):
    print(i)

In [ ]:
#check iterator again
for i in clf:
    print(i)

## Test STree is a sklearn estimator

In [ ]:
# Make checks one by one
c = 0
checks = check_estimator(Stree(), generate_only=True)
for check in checks:
    c += 1
    print(c, check[1])
    check[1](check[0])

In [ ]:
# Check if the classifier is a sklearn estimator
check_estimator(Stree())

## Compare to SVM

In [ ]:
svc = SVC(C=7, kernel='rbf', gamma=.001, random_state=random_state)
clf = Stree(C=17, kernel='rbf', gamma=.001, random_state=random_state)
svc.fit(Xtrain, ytrain)
clf.fit(Xtrain, ytrain)
print("== Not Weighted ===")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
svc.fit(Xtrain, ytrain, weights)
clf.fit(Xtrain, ytrain, weights)
print("==== Weighted =====")
print("SVC train score ..:", svc.score(Xtrain, ytrain))
print("STree train score :", clf.score(Xtrain, ytrain))
print("SVC test score ...:", svc.score(Xtest, ytest))
print("STree test score .:", clf.score(Xtest, ytest))
print("*SVC test score ..:", svc.score(Xtest, ytest, weights_test))
print("*STree test score :", clf.score(Xtest, ytest, weights_test))

In [ ]:
print(clf)

## Test max_features

In [ ]:
for max_features in [None, "auto", "log2", 7, .5, .1, .7]:
    now = time.time()
    print("*"*40)
    clf = Stree(random_state=random_state, max_features=max_features)
    clf.fit(Xtrain, ytrain)
    print(f"max_features {max_features} = {clf.max_features_}")
    print("Train score :", clf.score(Xtrain, ytrain))
    print("Test score .:", clf.score(Xtest, ytest))
    print(f"Took {time.time() - now:.2f} seconds")